In [16]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import bs4 as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [17]:
#get an url list 
def get_url_list(page1_url):
    var_url = '&section_offset=5&items_offset='
    url_lst = [page1_url]
    for page in range(1,17):
        url = page1_url+var_url+str(18*page)
        url_lst.append(url)
    return url_lst

In [18]:
def get_house_info(url, price):
    import requests
    from bs4 import BeautifulSoup
    import re
    response = requests.get(url)
    output = []
    if not response.status_code == 200:
        print('failed to get the fidelity website')
        return output
    else:
        results_page = BeautifulSoup(response.content,'html5lib')
        # rating
        if len(results_page.find_all('div', class_="_l0ao8q")) == 0:
            rating = 'no rating'
        else:
            rating_sec = results_page.find_all('div', class_="_l0ao8q")[0]
            rating = rating_sec.find('span').get('aria-label')
        # coordinates
        coordinates = re.search(
            '"listing_lat":([0-9]{2}.[0-9]*),"listing_lng":(-[0-9]{2}.[0-9]*),', 
            response.text)
        
        lat = coordinates.group(1)
        lon = coordinates.group(2)
        # host
        superhost = re.search('"is_superhost":(.{1,5})', 
                              response.text).group(1)
        
        if len(results_page.find_all('span', class_="_fgdupie"))==0:
            response_rate = "no info"
        else:
            host = results_page.find_all('span', class_="_fgdupie")
            #response rate
            response_rate = float('nan')
            for host_ in host[::-1]:
                if 'Response rate' in host_.text:
                    response_rate = host_.text.split(':')[-1]
                    break
                
        if len(results_page.find_all('span',class_="_fgdupie"))==0:
            accommodates = "no info"
            bedrooms = "no info"
            beds = "no info"
            bathrooms = "no info"
        else:
            #accomondates   
            accommodates = results_page.find_all('span',class_="_fgdupie")[0].text
            #number of bedrooms
            bedrooms = results_page.find_all('span',class_="_fgdupie")[1].text       
            #number of beds
            beds = results_page.find_all('span',class_="_fgdupie")[2].text       
            #number of bathrooms
            bathrooms = results_page.find_all('span',class_="_fgdupie")[3].text
        
        
        #room type
        try:
            room_type = results_page.find('span',class_="_1hh2h7tb").text
        except AttributeError:
            room_type='NoneType'

        #number of review
        try:
            number_of_reviews = results_page.find('div', class_="_vy3ibx").text
        except AttributeError:
            number_of_reviews='NoneType'
        
        output=[price, accommodates, bedrooms,beds, bathrooms, room_type, 
                number_of_reviews,  rating, lat, lon, superhost, response_rate]
        #print('output=')
        #print(output)
        
    return output

In [24]:
def get_prices_urls(url,dataframe):
    info_list=[]
    from selenium import webdriver  
    browser = webdriver.Chrome('/Users/mac/Desktop/chromedriver')
    #url='https://www.airbnb.com/s/Manhattan--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&toddlers=0&query=Manhattan%2C%20New%20York%2C%20NY%2C%20United%20States&allow_override%5B%5D=&price_min=100&price_max=199&s_tag=qfon6p2A'
    browser.get(url)
    html_source = browser.page_source
    #browser.quit()
    #get the prices for every apartment since the price infomation is not availabe inside the apt page
    prices = browser.find_elements(By.XPATH,"//span[@class='_17oldnte']")
    price_list = []
    for price in prices:
        try:
            price = price.text
            price_list.append(price)
        except:
            pass
    price_list = price_list[::3]
    browser.quit()
    print("length of price list is: ", len(price_list))

    #get the url for every apt on this page
    results_page = BeautifulSoup(html_source, "lxml")
    urls = results_page.find_all("meta", content=re.compile("www.airbnb.com/rooms"))
    #urls = browser.find_elements(By.XPATH,"//a[@class='_1ol0z3h']")
    url_list = []
    for url in urls:
        url = str(url)
        url = "https://"+url[15:-18]
        #print (x)
        url_list.append(url)
    print("length of apt_url_list is:" + str(len(url_list)))
    print("-----------------")
    #print(url_list)
    #pass url and price to the get_house_info function and extract the rest infomation 
    
    if (len(price_list) == len(url_list)) and (len(price_list)>0):
        for x in range(len(price_list)):
            #print(get_house_info(url_list[x], price_list[x]))
            info_list.append(get_house_info(url_list[x], price_list[x]))
        #print('info list is:')
        #print(info_list)
        df2=pd.DataFrame(info_list,columns=['price','accommodates', 'bedrooms','beds', 
                                            'bathrooms', 'room_type', 'number_of_reviews',  'rating', 
                                            'lat', 'lon', 'superhost', 'response_rate'])
        #print('dataframe is:')
        #print(df2)
        
        dataframe=pd.concat([dataframe,df2], ignore_index=True)
        print('complete dataframe')

    #print("info_list=")
    #print(info_list)
    return dataframe  

In [20]:
page_url="https://www.airbnb.com/s/Manhattan--New-York--NY--United-States/homes?refinement_paths%5B%5D=%2Fhomes&allow_override%5B%5D=&locale=en&ne_lat=40.81728485589226&ne_lng=-73.89477434326723&price_max=308&price_min=250&query=Manhattan%2C%20New%20York%2C%20NY%2C%20United%20States&search_by_map=true&sw_lat=40.69466178753095&sw_lng=-74.04491647504908&toddlers=0&zoom=12&map_toggle=true&s_tag=c16viQaR"
url_list=get_url_list(page_url)
df = pd.DataFrame(columns=['price','accommodates', 'bedrooms','beds', 'bathrooms', 
                               'room_type', 'number_of_reviews',  'rating', 'lat', 'lon', 
                               'superhost', 'response_rate'])
for url in url_list:
    df=get_prices_urls(url, df)

length of price list is:  18
length of apt_url_list is:0
-----------------
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  0
length of apt_url_list is:0
-----------------
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  0
length of apt_url_list is:0
-----------------
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe
length of price list is:  0
length of apt_url_list is:0
-----------------
length of price list is:  0
length of apt_url_list is:0
-----------------
length of price list is:  18
length of apt_url_list is:18
-----------------
complete dataframe


In [23]:
len(df['lat'].unique())

72